In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#https://towardsdatascience.com/end-to-end-case-study-bike-sharing-demand-dataset-53201926c8db
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/bikesfortutorial/Bikes.csv')

In [ ]:
df.head()

In [ ]:
df.rename(columns = {'hum':'humidity','weathersit':'weather','cnt':'count'},inplace = True)

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
from datetime import datetime
time = []
month = []
for i in df['datetime']:
    dt_object2 = datetime.strptime(i, "%Y-%m-%d %H:%M:%S")
    time.append(dt_object2.hour)
    month.append(dt_object2.month)
df['time'] = pd.DataFrame(time)
df['time'] = df['time'].astype(float)
df['month'] = pd.DataFrame(month)
df['month'] = df['month'].astype(float)

In [ ]:
df.drop('datetime',axis=1,inplace=True) 
df.drop('holiday',axis=1,inplace=True) 
df = df.drop('atemp',axis=1)
df = pd.get_dummies(df,columns=['season','weather'],drop_first=True)

In [ ]:
for i in df.columns:
    df[i].fillna(value = df[i].median())

In [ ]:
df

In [ ]:
 work_day = df[df['workingday']==1] 
non_work_day = df[df['workingday']==0]
    # Model for registered
X = work_day.drop(['casual','registered','count'],axis = 1)
y = work_day.registered
    # Dividing the data into train and test


In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.30 ,random_state = 2)

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(x_train,y_train)
y_pred = regressor.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_pred,y_test)
rmse=np.sqrt(mse)
print('RMLSE for the data:',rmse)

In [ ]:
# Finding best parameters for decision tree
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(random_state=0)
dt_params = {'max_depth':np.arange(1,50,2),'min_samples_leaf':np.arange(2,15)}
    
from sklearn.model_selection import GridSearchCV
gs_dt = GridSearchCV(dt,dt_params,cv=3)
gs_dt.fit(x_train,y_train)
a = gs_dt.best_params_

In [ ]:
a

In [ ]:
# Training with best parameters
from sklearn.tree import DecisionTreeRegressor
dtr=DecisionTreeRegressor(max_depth=a['max_depth'],min_samples_leaf= a['min_samples_leaf'])
model = dtr.fit(x_train,y_train)
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error(y_pred,y_test)
rmsle=np.sqrt(msle)
print('RMLSE for the data:',rmsle) # For decision tree

from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_pred,y_test)
rmse=np.sqrt(mse)
print('RMLSE for the data:',rmse)

In [ ]:
# Plotting important features
import matplotlib.pyplot as plt
importances = dtr.feature_importances_
plt.title('Registered Feature Importances')
plt.barh(range(len(importances)), importances, color='g', align='center')
plt.yticks(range(len(importances)), X.columns)
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Finding best parameters for RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state=0)
rf_params = {'n_estimators':np.arange(25,150,25),'max_depth':np.arange(1,11,2),'min_samples_leaf':np.arange(2,15,3)}

from sklearn.model_selection import GridSearchCV
gs_rf = GridSearchCV(rf,rf_params,cv=3)
gs_rf.fit(x_train,y_train)
b = gs_rf.best_params_

In [ ]:
# Fitting the model with best params
RF = RandomForestRegressor(n_estimators=b['n_estimators'],max_depth=b['max_depth'],min_samples_leaf=b['min_samples_leaf'],random_state=0)
model = RF.fit(x_train,y_train)
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error(y_pred,y_test)
rmsle=np.sqrt(msle)
print('RMLSE for the data:',rmsle) # For random forest

In [ ]:
# Plotting important features
import matplotlib.pyplot as plt
importances = RF.feature_importances_
plt.title('Registered Feature Importances')
plt.barh(range(len(importances)), importances, color='g', align='center')
plt.yticks(range(len(importances)), X.columns)
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
x = non_work_day.drop(['casual','registered','count'],axis = 1)
y = non_work_day.casual
    # Dividing the data into train and test
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.30 ,random_state = 2)

from sklearn.ensemble import AdaBoostRegressor
ar = AdaBoostRegressor(base_estimator=RF,random_state=0)
ar_params = {'n_estimators':np.arange(25,200,25)}
from sklearn.model_selection import GridSearchCV
gs_ar = GridSearchCV(ar,ar_params,cv=3)
gs_ar.fit(x_train,y_train)
c = gs_ar.best_params_

ab_rf = AdaBoostRegressor(base_estimator=RF,n_estimators=c['n_estimators'],random_state=0)
model = ab_rf.fit(x_train,y_train)
y_pred = model.predict(x_test)

from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error(y_pred,y_test)
rmsle=np.sqrt(msle)
print('RMLSE for the data:',rmsle) # For Ada-Boost
